In [1]:
from collections import defaultdict
import uuid
import time
import heapq
import datetime


In [2]:
class Order:
    def __init__(self,instrument,order_id,side,price,quantity,timestamp):
        self.instrument = instrument
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.timestamp = timestamp
        self.filled_quantity = 0
        
    # price/time priority
    def __lt__(self,other):
        if self.side == 'buy':
            return (self.price, -self.timestamp) > (other.price,-other.timestamp) # to get most recent buy order
        else:
            return (self.price,self.timestamp) < (other.price, other.timestamp)
           

In [25]:
class OrderBook:
    def __init__(self):
        self.bids = []
        self.asks = []
        
    def add(self,order):
        if order.side =='buy':
            heapq.heappush(self.bids,(-order.price,order.timestamp,order))
        elif order.side == 'sell':
            heapq.heappush(self.asks,(order.price,order.timestamp,order))
            
            
    def remove(self, order):
        if order.side == 'buy':
            self.bids.remove((-order.price,order.timestamp,order))
            heapq.heapify(self.bids)
        else:
            self.asks.remove((order.price,order.timestamp,order))
            heapq.heapify(self.asks)
            
    def bestBid(self):
        return self.bids[0][2] if self.bids else None

    def bestAsk(self):
        return self.asks[0][2] if self.asks else None
    
    def showBestBidAndAsk(self):
        best_bid = self.bestBid()
        best_ask = self.bestAsk()

        if (best_bid is not None and best_bid.quantity>0):
            print(f"Best Bid: Instrument: {best_bid.instrument}, Price: {best_bid.price}, Quantity: {best_bid.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best bid available.")
            print("----------------------------------------------------------------------------------------------------")

        if (best_ask is not None and best_ask.quantity>0):
            print(f"Best Ask: Instrument: {best_ask.instrument}, Price: {best_ask.price}, Quantity: {best_ask.quantity}")
            print("----------------------------------------------------------------------------------------------------")
        else:
            print("No best ask available.")
            print("----------------------------------------------------------------------------------------------------")

In [26]:
class Trade:
    def __init__(self,id,name,price,volume):
        self.id = id
        self.name = name
        self.price = price
        self.volume = volume

In [27]:
class MatchedOrder:
    def __init__(self, order_id, side, price, filled_quantity, instrument, timestamp):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.filled_quantity = filled_quantity
        self.instrument = instrument
        self.timestamp = timestamp

In [32]:
class MatchingEngine:
    def __init__(self):
        self.orderbooks = defaultdict(OrderBook)
        self.trades = defaultdict(list)
        self.buyerAckQueue = []
        self.sellerAckQueue = []
        self.orders = {}
        
    def placeOrder(self,instrument,side,price,quantity):
        order = Order(instrument,uuid.uuid1(),side,price,quantity,timestamp=int(time.time()))
        self.processOrder(order)
        self.orders[order.order_id] = order
        return order.order_id
    
    def getOrderBook(self,instrument):
        orderbook = self.orderbooks[instrument]
        # orderbook.showBestBidAndAsk()
        return orderbook.bids,orderbook.asks
    
    def showBestBidAsk(self,instrument):
        orderbook = self.orderbooks[instrument]
        orderbook.showBestBidAndAsk()
        
        
        
    
    def getTrades(self,instrument):
        return self.trades[instrument]
       
    def processOrder(self,order):
            orderbook = self.orderbooks[order.instrument]
            
            # add it orderbook 
            if (order.quantity >0):
                orderbook.add(order)
             
            if (order.side == 'buy'):
                print(f"Buy order {order.order_id,order.instrument,order.price,order.quantity} is added to orderbook")
                
                
            else:
                print(f"Sale order {order.order_id,order.instrument,order.price,order.quantity} is added to orderbook")
        
            if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
        # Buy order crossed the spread.
                self.matchBuyOrder(order, orderbook)
                
                
            elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
        # Sell order crossed the spread.
                self.matchSellOrder(order, orderbook)
                print(f"Sale order {order.order_id,order.instrument,order.price} matched with buy order {orderbook.bestBid().order_id,orderbook.bestBid().price}")
                print("----------------------------------------------------------------------------------------------------")
            else:
                # Order did not cross the spread, place in order book
                return None
                
                    
        
    def acknowledgeOrder(self, matchedOrder):
        
        timeACk = datetime.datetime.fromtimestamp(matchedOrder.timestamp)
        # Print the order details
        print(f"Order acknowledgmenet for buy order {matchedOrder.order_id,matchedOrder.instrument}")
        print(f"Order ID: {matchedOrder.order_id}")
        print(f"Instrument: {matchedOrder.instrument}")
        print(f"Price: {matchedOrder.price}")
        print(f"Quantity: {matchedOrder.filled_quantity}")
        print(f"Timestamp: {timeACk}")
        print(f"Action: {'Bought' if matchedOrder.side == 'buy' else 'Sold'}")
        print("----------------------------------------------------------------------------------------------------")
        
        if matchedOrder.side == 'buy':
            self.buyerAckQueue.append(matchedOrder)
        else:
            self.sellerAckQueue.append(matchedOrder)
        
    def cancelOrder(self, order_id):
        if order_id in self.orders:
            order = self.orders[order_id]
            if order.filled_quantity == order.quantity:
                print(f"Cannot cancel order {order_id}, already fully filled")
                print("----------------------------------------------------------------------------------------------------")
                self.buyerAckQueue.append(order)  # Send the order to the buyer
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]
                return True
            elif order.filled_quantity < order.quantity:
                remaining_quantity = order.quantity - order.filled_quantity
                order.quantity -= order.filled_quantity
                orderbook = self.orderbooks[order.instrument]
                orderbook.remove(order)
                del self.orders[order_id]

                # Print the message with filled and remaining quantities
                print(f"Order ID: {order_id} is cancelled. Filled quantity: {order.filled_quantity}, Remaining quantity: {remaining_quantity}")
                print("----------------------------------------------------------------------------------------------------")
                return True
            else:
                return False

        else:
            print("There is no such order")
            print("----------------------------------------------------------------------------------------------------")
            return False

                
    def match(self,order):
        orderbook = self.orderbooks[order.instrument]
        
        if (order.side == 'buy' and orderbook.bestAsk() is not None and order.price >= orderbook.bestAsk().price):
    # Buy order crossed the spread.
            self.matchBuyOrder(order, orderbook)
        elif (order.side == 'sell' and orderbook.bestBid() is not None and order.price <= orderbook.bestBid().price):
    # Sell order crossed the spread.
            self.matchSellOrder(order, orderbook)
        # else:
        #     # # Order did not cross the spread, place in order book
        #     orderbook.add(order)
            
        
    def matchBuyOrder(self,order, orderbook):
        while orderbook.asks and order.price >= orderbook.bestAsk().price and order.quantity > 0 :
            ask = orderbook.bestAsk()
            
            if ask.quantity <= order.quantity:
                order.quantity -= ask.quantity
                order.filled_quantity += ask.quantity
                trade = Trade(ask.order_id,ask.instrumenet,ask.price,ask.quantity)
                self.trades[order.instrument].append(trade)
                self.acknowledgeOrder(MatchedOrder(ask.order_id,'sell',ask.price,ask.quantity,order.instrument,ask.timestamp))
                orderbook.remove(ask)
            else: 
                trade = Trade(ask.order_id,ask.instrumenet,ask.price,ask.quantity)
                self.trades[order.instrument].append(trade)
                ask.quantity -= order.quantity
                self.acknowledgeOrder(MatchedOrder(ask.order_id,'sell',ask.price,order.quantity,order.instrument,ask.timestamp))
                order.filled_quantity += order.quantity
                order.quantity = 0
                
                
    def matchSellOrder(self, order, orderbook):
        while orderbook.bids and order.price >= orderbook.bestBid().price and order.quantity > 0:
            bid = orderbook.bestBid()

            if bid.quantity <= order.quantity:
                order.quantity -= bid.quantity
                order.filled_quantity += bid.quantity
                trade = Trade(bid.order_id,bid.price,bid.price, bid.quantity)
                self.trades[order.instrument].append(trade)
                self.acknowledgeOrder(MatchedOrder(bid.order_id, 'sell', bid.price, bid.quantity, order.instrument, bid.timestamp))
                orderbook.remove(bid)
            else:
                trade = Trade(bid.order_id,bid.price,bid.price, bid.quantity)
                self.trades[order.instrument].append(trade)
                bid.quantity -= order.quantity
                self.acknowledgeOrder(MatchedOrder(bid.order_id, 'sell', bid.price, order.quantity, order.instrument, bid.timestamp))
                order.filled_quantity += order.quantity
                order.quantity = 0

        # If the sell order is not fully filled, add the remaining part to the order book
        if order.quantity > 0:
            orderbook.add(order)
        
    

### TESTING OUTPUT (DEBUGGING)

In [29]:
# my_order = Order('ETH', 12345,'buy', 150.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# my_order2 = Order('ETH', 145,'buy', 230.0, 100, 1648763456)
# print("Instrument:", my_order.instrument)
# print("Order ID:", my_order.order_id)
# print("Side:", my_order.side)
# print("Price:", my_order.price)
# print("Quantity:", my_order.quantity)
# print("Timestamp:", my_order.timestamp)
# print("Filled Quantity:", my_order.filled_quantity)

# myOrderBook = OrderBook()
# myOrderBook.add(my_order)
# myOrderBook.add(my_order2)
# ord = myOrderBook.bestBid()
# print(ord.order_id)

### TEST CASES

#### Place buy order n retrieve orderbook

In [33]:
instrument = "ETH"

In [34]:
# Create a matching engine instance.
engine = MatchingEngine()

# Place orders.
buy_order_id1 = engine.placeOrder("ETH", "buy", 150, 5)
#buy_order_id2 = engine.placeOrder("BTC", "buy", 160, 5)

#buy_order_id3 = engine.placeOrder("BTC", "buy", 250, 3)

buy_order_id4 = engine.placeOrder("ETH", "buy", 120, 5)
buy_order_id5 = engine.placeOrder("ETH", "buy", 110, 5)
sell_order_id1 = engine.placeOrder("ETH", "sell", 150, 2)




bids, asks = engine.getOrderBook(instrument)
print("----------------------------------------------------------------------------------------------------")
print(f"OrderBook for {instrument}")
print("Bids:", bids)
print("Asks:", asks)
print("----------------------------------------------------------------------------------------------------")

engine.showBestBidAsk("ETH")

Buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 'ETH', 150, 5) is added to orderbook
Buy order (UUID('b2156ddf-e600-11ed-9867-9078418aa6e1'), 'ETH', 120, 5) is added to orderbook
Buy order (UUID('b2156de0-e600-11ed-b244-9078418aa6e1'), 'ETH', 110, 5) is added to orderbook
Sale order (UUID('b2156de1-e600-11ed-8eca-9078418aa6e1'), 'ETH', 150, 2) is added to orderbook
Order acknowledgmenet for buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 'ETH')
Order ID: b21546c9-e600-11ed-b59e-9078418aa6e1
Instrument: ETH
Price: 150
Quantity: 2
Timestamp: 2023-04-29 04:10:18
Action: Sold
----------------------------------------------------------------------------------------------------
Sale order (UUID('b2156de1-e600-11ed-8eca-9078418aa6e1'), 'ETH', 150) matched with buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 150)
----------------------------------------------------------------------------------------------------
--------------------------------------------------------

#### Place sell order n retrieve orderbook

In [35]:
#engine = MatchingEngine()

# Place orders.

sell_order_id1 = engine.placeOrder("ETH", "sell", 150, 2)

sell_order_id2 = engine.placeOrder("ETH", "sell", 120, 3)
#sell_order_id3 = engine.placeOrder("BTC", "sell", 150, 2)
#sell_order_id4= engine.placeOrder("BTC", "sell", 190, 5)



# buy_order_id = engine.placeOrder("ETH", "buy", 120, 10)
# sell_order_id = engine.placeOrder("ETH", "sell", 132, 5)
# Get the order book.
bids, asks = engine.getOrderBook(instrument)
print("----------------------------------------------------------------------------------------------------")
print(f"OrderBook for {instrument}")
print("Bids:", bids)
print("Asks:", asks)
print("----------------------------------------------------------------------------------------------------")

engine.showBestBidAsk("ETH")

Sale order (UUID('b49384cc-e600-11ed-8b73-9078418aa6e1'), 'ETH', 150, 2) is added to orderbook
Order acknowledgmenet for buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 'ETH')
Order ID: b21546c9-e600-11ed-b59e-9078418aa6e1
Instrument: ETH
Price: 150
Quantity: 2
Timestamp: 2023-04-29 04:10:18
Action: Sold
----------------------------------------------------------------------------------------------------
Sale order (UUID('b49384cc-e600-11ed-8b73-9078418aa6e1'), 'ETH', 150) matched with buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 150)
----------------------------------------------------------------------------------------------------
Sale order (UUID('b493ab21-e600-11ed-8895-9078418aa6e1'), 'ETH', 120, 3) is added to orderbook
Sale order (UUID('b493ab21-e600-11ed-8895-9078418aa6e1'), 'ETH', 120) matched with buy order (UUID('b21546c9-e600-11ed-b59e-9078418aa6e1'), 150)
--------------------------------------------------------------------------------------------------

#### Get Trades done

In [10]:


# Get trades.
trades = engine.getTrades("ETH")
print("Trades:", trades)

# Get filled orders.
if engine.buyerAckQueue:
    buy_ack = engine.buyerAckQueue.pop()
    print("Buy filled:", buy_ack.order_id, buy_ack.side, buy_ack.price, buy_ack.filled_quantity)
else:
    print("No buy acknowledgements")

if engine.sellerAckQueue:
    sell_ack = engine.sellerAckQueue.pop()
    print("Sell filled:", sell_ack.order_id, sell_ack.side, sell_ack.price, sell_ack.filled_quantity)
else:
    print("No sell acknowledgements")


Trades: []
No buy acknowledgements
No sell acknowledgements
